<a href="https://colab.research.google.com/github/rajveer43/titan_transformer/blob/master/Google_Titan_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Titan Model 1

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DepthwiseSeparableConv1D(nn.Module):
    def __init__(self, in_channels, kernel_size):
        super().__init__()
        self.depthwise = nn.Conv1d(in_channels, in_channels, kernel_size, groups=in_channels, padding=kernel_size // 2)
        self.pointwise = nn.Conv1d(in_channels, in_channels, kernel_size=1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class GatingMechanism(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, input_dim)

    def forward(self, x):
        gate = torch.sigmoid(self.linear(x))
        return gate * x

class TitansMemoryModule(nn.Module):
    def __init__(self, input_dim, memory_dim):
        super().__init__()
        self.long_term_memory = nn.Linear(input_dim, memory_dim)
        self.persistent_memory = nn.Parameter(torch.zeros(memory_dim))

    def forward(self, x):
        long_term_output = self.long_term_memory(x)
        combined_memory = long_term_output + self.persistent_memory.unsqueeze(0)
        return combined_memory

class TitansAttention(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.query = nn.Linear(input_dim, input_dim)
        self.key = nn.Linear(input_dim, input_dim)
        self.value = nn.Linear(input_dim, input_dim)

        self.conv = DepthwiseSeparableConv1D(input_dim, kernel_size=3)
        self.gating = GatingMechanism(input_dim)

    def forward(self, x):
        q = F.normalize(self.query(x), p=2, dim=-1)
        k = F.normalize(self.key(x), p=2, dim=-1)
        v = self.value(x)

        attention_weights = torch.matmul(q, k.transpose(-2, -1)) / torch.sqrt(torch.tensor(x.size(-1), dtype=torch.float32))
        attention_weights = F.softmax(attention_weights, dim=-1)
        attention_output = torch.matmul(attention_weights, v)

        attention_output = attention_output.transpose(1, 2)  # Prepare for Conv1D
        conv_output = self.conv(attention_output)
        conv_output = conv_output.transpose(1, 2)  # Back to original shape

        gated_output = self.gating(conv_output)
        return gated_output

class TitansModel(nn.Module):
    def __init__(self, input_dim, memory_dim):
        super().__init__()
        self.memory_module = TitansMemoryModule(input_dim, memory_dim)
        self.attention = TitansAttention(memory_dim)
        self.residual = nn.Identity()

    def forward(self, x):
        memory_output = self.memory_module(x)
        attention_output = self.attention(memory_output)
        return self.residual(attention_output + memory_output)

# Example usage
input_dim = 128
memory_dim = 256
sequence_length = 32
batch_size = 8

model = TitansModel(input_dim, memory_dim)
inputs = torch.randn(batch_size, sequence_length, input_dim)
outputs = model(inputs)
print(outputs.shape)


torch.Size([8, 32, 256])


# Titan implementation

## Titan Model Implementation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class DepthwiseSeparableConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super().__init__()
        self.depthwise = nn.Conv1d(
            in_channels, in_channels, kernel_size,
            padding=padding, groups=in_channels
        )
        self.pointwise = nn.Conv1d(in_channels, out_channels, 1)

    def forward(self, x):
        x = self.depthwise(x)
        x = self.pointwise(x)
        return x

class GatingMechanism(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.gate_proj = nn.Linear(dim, dim)
        self.transform_proj = nn.Linear(dim, dim)

    def forward(self, x):
        gate = torch.sigmoid(self.gate_proj(x))
        transformed = self.transform_proj(x)
        return gate * transformed

class TitanAttention(nn.Module):
    def __init__(self, dim, num_heads=8, head_dim=64, dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = head_dim
        self.scale = head_dim ** -0.5

        # Projections for Q, K, V
        self.q_proj = nn.Linear(dim, num_heads * head_dim)
        self.k_proj = nn.Linear(dim, num_heads * head_dim)
        self.v_proj = nn.Linear(dim, num_heads * head_dim)

        # Depthwise separable convolutions
        self.q_conv = DepthwiseSeparableConv1d(
            num_heads * head_dim,
            num_heads * head_dim,
            kernel_size=3,
            padding=1
        )
        self.k_conv = DepthwiseSeparableConv1d(
            num_heads * head_dim,
            num_heads * head_dim,
            kernel_size=3,
            padding=1
        )
        self.v_conv = DepthwiseSeparableConv1d(
            num_heads * head_dim,
            num_heads * head_dim,
            kernel_size=3,
            padding=1
        )

        self.out_proj = nn.Linear(num_heads * head_dim, dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        batch_size, seq_len, dim = x.shape

        # Project and apply SiLU activation
        q = F.silu(self.q_proj(x))
        k = F.silu(self.k_proj(x))
        v = F.silu(self.v_proj(x))

        # Reshape for depthwise conv
        q = q.transpose(1, 2)
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)

        # Apply depthwise separable convolutions
        q = self.q_conv(q).transpose(1, 2)
        k = self.k_conv(k).transpose(1, 2)
        v = self.v_conv(v).transpose(1, 2)

        # Reshape for multi-head attention
        q = q.reshape(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.reshape(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.reshape(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        # L2 normalize queries and keys
        q = F.normalize(q, p=2, dim=-1)
        k = F.normalize(k, p=2, dim=-1)

        # Compute attention scores
        attn = torch.matmul(q, k.transpose(-2, -1)) * self.scale

        if mask is not None:
            # Properly reshape mask for broadcasting
            # mask shape: [batch_size, seq_len] -> [batch_size, 1, 1, seq_len]
            mask = mask.unsqueeze(1).unsqueeze(2)
            attn = attn.masked_fill(mask == 0, float('-inf'))

        attn = F.softmax(attn, dim=-1)
        attn = self.dropout(attn)

        # Apply attention to values
        out = torch.matmul(attn, v)

        # Reshape and project output
        out = out.transpose(1, 2).reshape(batch_size, seq_len, -1)
        out = self.out_proj(out)

        return out

class TitanBlock(nn.Module):
    def __init__(self, dim, num_heads=8, head_dim=64, mlp_ratio=4, dropout=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = TitanAttention(dim, num_heads, head_dim, dropout)
        self.norm2 = nn.LayerNorm(dim)

        mlp_dim = int(dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.SiLU(),
            GatingMechanism(mlp_dim),
            nn.Dropout(dropout),
            nn.Linear(mlp_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x, mask=None):
        # Residual connection for attention
        x = x + self.attn(self.norm1(x), mask)
        # Residual connection for MLP
        x = x + self.mlp(self.norm2(x))
        return x

class TitanTransformer(nn.Module):
    def __init__(
        self,
        dim,
        depth,
        num_heads=8,
        head_dim=64,
        mlp_ratio=4,
        dropout=0.1
    ):
        super().__init__()
        self.layers = nn.ModuleList([
            TitanBlock(
                dim=dim,
                num_heads=num_heads,
                head_dim=head_dim,
                mlp_ratio=mlp_ratio,
                dropout=dropout
            )
            for _ in range(depth)
        ])

        self.norm = nn.LayerNorm(dim)

    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

# Example usage
def create_titan_model(
    vocab_size=50000,
    max_seq_length=1024,
    dim=512,
    depth=12,
    num_heads=8,
    head_dim=64,
    mlp_ratio=4,
    dropout=0.1
):
    class TitanModel(nn.Module):
        def __init__(self):
            super().__init__()
            self.embedding = nn.Embedding(vocab_size, dim)
            self.pos_embedding = nn.Parameter(torch.zeros(1, max_seq_length, dim))
            self.transformer = TitanTransformer(
                dim=dim,
                depth=depth,
                num_heads=num_heads,
                head_dim=head_dim,
                mlp_ratio=mlp_ratio,
                dropout=dropout
            )

        def forward(self, x, mask=None):
            # Add positional embeddings
            x = self.embedding(x)
            x = x + self.pos_embedding[:, :x.size(1), :]

            # Apply transformer
            x = self.transformer(x, mask)
            return x

    return TitanModel()

In [ ]:
# Create a model
model = create_titan_model(
    vocab_size=50000,
    max_seq_length=1024,
    dim=512,
    depth=12
)

# Example input (batch_size=2, seq_length=128)
x = torch.randint(0, 50000, (2, 128))
mask = torch.ones(2, 128)

# Forward pass
output = model(x, mask)

In [ ]:
print(output)

tensor([[[-2.5337e-01, -6.0905e-01,  2.3120e+00,  ..., -2.9865e-01,
          -1.7289e-01,  1.9123e+00],
         [-7.7001e-01,  1.0869e+00,  3.5012e+00,  ...,  1.3517e+00,
          -6.5708e-02, -4.5865e-02],
         [-1.3881e+00,  8.3592e-02,  1.2183e-01,  ..., -1.8826e+00,
          -1.4912e-01, -9.6813e-01],
         ...,
         [-1.6125e+00,  2.4097e-01,  1.0992e+00,  ..., -2.0965e-01,
           1.4801e-01,  1.5081e-02],
         [ 4.3039e-01,  7.8977e-01,  1.8156e+00,  ..., -6.9191e-02,
          -2.0223e+00, -4.6822e-01],
         [-1.2896e+00, -7.1673e-01,  6.4670e-01,  ..., -4.1999e-01,
          -9.2779e-01, -2.3601e-01]],

        [[-8.0844e-02,  6.1648e-01,  2.4976e+00,  ...,  7.4161e-01,
          -1.3234e+00, -2.1923e-01],
         [-9.1315e-01,  2.2983e-01,  9.5114e-01,  ...,  1.0183e+00,
          -1.2915e+00, -5.7810e-04],
         [-5.8095e-01, -4.8502e-02,  1.2166e+00,  ..., -7.5621e-01,
          -2.1875e-01,  2.9964e-01],
         ...,
         [-1.7111e+00,  1

## using the huggingface tokenizer

In [ ]:
import torch
import torch.nn.functional as F
from typing import List, Optional, Tuple
from transformers import PreTrainedTokenizerFast
from transformers import GPT2Tokenizer

class TitanTextGenerator:
    def __init__(
        self,
        model: 'TitanModel',
        tokenizer: Optional[PreTrainedTokenizerFast] = None,
        max_length: int = 1024,
        device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    ):
        self.model = model.to(device)
        self.model.eval()  # Set to evaluation mode
        self.device = device
        self.max_length = max_length

        # Initialize tokenizer if not provided
        if tokenizer is None:
            self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            self.tokenizer.pad_token = self.tokenizer.eos_token
        else:
            self.tokenizer = tokenizer

    def preprocess_text(self, text: str) -> Tuple[torch.Tensor, torch.Tensor]:
        # Tokenize input text
        encodings = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        input_ids = encodings['input_ids'].to(self.device)
        attention_mask = encodings['attention_mask'].to(self.device)

        return input_ids, attention_mask

    @torch.no_grad()
    def generate_next_token(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        temperature: float = 1.0,
        top_k: int = 50,
        top_p: float = 0.9
    ) -> torch.Tensor:
        # Get model output
        outputs = self.model(input_ids, attention_mask)

        # Get logits for the next token (last position)
        next_token_logits = outputs[:, -1, :]

        # Apply temperature
        next_token_logits = next_token_logits / temperature

        # Apply top-k filtering
        if top_k > 0:
            values, indices = torch.topk(next_token_logits, top_k)
            min_values = values[:, -1].unsqueeze(-1).expand_as(next_token_logits)
            next_token_logits = torch.where(
                next_token_logits < min_values,
                torch.ones_like(next_token_logits) * float('-inf'),
                next_token_logits
            )

        # # Apply top-p (nucleus) filtering
        # if top_p < 1.0:
        #     # Sort logits in descending order
        #     sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True, dim=-1)
        #     cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        #     # Create a mask for tokens to remove
        #     sorted_indices_to_remove = cumulative_probs > top_p

        #     # Shift the indices to the right to keep also the first token above the threshold
        #     sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        #     sorted_indices_to_remove[..., 0] = 0

        #     # Scatter sorted indices to original positions
        #     indices_to_remove = sorted_indices_to_remove.scatter(
        #         dim=1,
        #         index=sorted_indices,
        #         src=sorted_indices_to_remove
        #     )
        #     next_token_logits = next_token_logits.masked_fill(indices_to_remove, float('-inf'))

        # Apply top-p (nucleus) filtering
        if top_p < 1.0:
            # Sort logits in descending order
            sorted_logits, sorted_indices = torch.sort(next_token_logits, descending=True, dim=-1)
            cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

            # Mask tokens with cumulative probability above top_p
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0  # Keep at least one token

            # Scatter mask back to logits
            mask = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
            next_token_logits = next_token_logits.masked_fill(mask, float('-inf'))


        # Sample from the filtered distribution
        probs = F.softmax(next_token_logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)

        return next_token

    def generate(
        self,
        text: str,
        max_new_tokens: int = 50,
        temperature: float = 1.0,
        top_k: int = 50,
        top_p: float = 0.9,
        early_stopping: bool = True
    ) -> str:
        # Preprocess input text
        input_ids, attention_mask = self.preprocess_text(text)

        generated_tokens = []

        for _ in range(max_new_tokens):
            # Generate next token
            next_token = self.generate_next_token(
                input_ids,
                attention_mask,
                temperature,
                top_k,
                top_p
            )

            # Append the new token
            generated_tokens.append(next_token.item())

            # Update input_ids and attention_mask
            input_ids = torch.cat([input_ids, next_token], dim=1)
            attention_mask = torch.cat([
                attention_mask,
                torch.ones((1, 1), device=self.device)
            ], dim=1)

            # Check for EOS token
            if early_stopping and next_token.item() == self.tokenizer.eos_token_id:
                break

            # Check if we've exceeded max_length
            if input_ids.size(1) >= self.max_length:
                break

        # Decode and return the complete text
        full_text = self.tokenizer.decode(
            input_ids[0].tolist(),
            skip_special_tokens=True
        )

        return full_text

# Example usage function
def create_text_generator(
    vocab_size=50000,
    max_seq_length=1024,
    dim=512,
    depth=12,
    device='cuda' if torch.cuda.is_available() else 'cpu'
):
    # Create the base model
    model = create_titan_model(
        vocab_size=vocab_size,
        max_seq_length=max_seq_length,
        dim=dim,
        depth=depth
    )

    # Create the generator
    generator = TitanTextGenerator(
        model=model,
        max_length=max_seq_length,
        device=device
    )

    return generator

In [ ]:
# Create the generator
generator = create_text_generator(
    vocab_size=50000,
    max_seq_length=1024,
    dim=512,
    depth=12
)

# Generate text
input_text = "MS thoni is world's best "
generated_text = generator.generate(
    text=input_text,
    max_new_tokens=50,
    temperature=0.3,  # Lower for more focused/deterministic output
    top_k=50,        # Number of highest probability vocabulary tokens to keep
    top_p=0.9        # Cumulative probability threshold for nucleus sampling
)

print(generated_text)

MS thoni is world's best  toil� I�artE O�� areage�ect that�ate Testopimic} lered�al'�rouimic�� stL it who(p�2 do! 2


## using custom tokenizer

In [ ]:
import torch
import torch.nn as nn
from collections import Counter
from typing import List, Dict, Tuple
import re
import numpy as np
from torch.nn.utils.rnn import pad_sequence

class CustomTokenizer:
    def __init__(
        self,
        vocab_size: int = 50000,
        min_freq: int = 2,
        special_tokens: List[str] = ["<PAD>", "<UNK>", "<BOS>", "<EOS>"]
    ):
        self.vocab_size = vocab_size
        self.min_freq = min_freq
        self.special_tokens = special_tokens

        # Initialize special token IDs
        self.pad_token_id = 0
        self.unk_token_id = 1
        self.bos_token_id = 2
        self.eos_token_id = 3

        # Initialize vocabularies
        self.token2idx: Dict[str, int] = {token: idx for idx, token in enumerate(special_tokens)}
        self.idx2token: Dict[int, str] = {idx: token for idx, token in enumerate(special_tokens)}
        self.vocab_size_current = len(special_tokens)

        # Regex for tokenization
        self.token_pattern = re.compile(r'\w+|[^\w\s]')

    def train_from_texts(self, texts: List[str]) -> None:
        """Train tokenizer on a list of texts."""
        # Count word frequencies
        word_counts = Counter()

        for text in texts:
            tokens = self._basic_tokenize(text)
            word_counts.update(tokens)

        # Filter by minimum frequency and vocab size
        filtered_tokens = [
            token for token, count in word_counts.most_common()
            if count >= self.min_freq and token not in self.special_tokens
        ]

        # Add tokens to vocabulary up to vocab_size
        remaining_space = self.vocab_size - len(self.special_tokens)
        for token in filtered_tokens[:remaining_space]:
            self.token2idx[token] = self.vocab_size_current
            self.idx2token[self.vocab_size_current] = token
            self.vocab_size_current += 1

    def _basic_tokenize(self, text: str) -> List[str]:
        """Basic tokenization into words and punctuation."""
        return self.token_pattern.findall(text.lower())

    def encode(self, text: str, add_special_tokens: bool = True) -> List[int]:
        """Encode text to token ids."""
        tokens = self._basic_tokenize(text)

        ids = []
        if add_special_tokens:
            ids.append(self.bos_token_id)

        for token in tokens:
            ids.append(self.token2idx.get(token, self.unk_token_id))

        if add_special_tokens:
            ids.append(self.eos_token_id)

        return ids

    def decode(self, ids: List[int], skip_special_tokens: bool = True) -> str:
        """Decode token ids back to text."""
        tokens = []
        for idx in ids:
            token = self.idx2token.get(idx, "<UNK>")
            if skip_special_tokens and token in self.special_tokens:
                continue
            tokens.append(token)
        return " ".join(tokens)

    def save_vocab(self, path: str) -> None:
        """Save vocabulary to file."""
        with open(path, 'w', encoding='utf-8') as f:
            for token, idx in sorted(self.token2idx.items(), key=lambda x: x[1]):
                f.write(f"{token}\t{idx}\n")

    def load_vocab(self, path: str) -> None:
        """Load vocabulary from file."""
        self.token2idx.clear()
        self.idx2token.clear()
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                token, idx = line.strip().split('\t')
                idx = int(idx)
                self.token2idx[token] = idx
                self.idx2token[idx] = token
        self.vocab_size_current = len(self.token2idx)

class CustomEmbedding(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        embedding_dim: int,
        pad_idx: int = 0,
        max_norm: float = None
    ):
        super().__init__()
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embedding_dim,
            padding_idx=pad_idx,
            max_norm=max_norm
        )
        self.embedding_dim = embedding_dim

        # Initialize embeddings using Xavier uniform initialization
        nn.init.xavier_uniform_(self.embedding.weight)
        with torch.no_grad():
            self.embedding.weight[pad_idx].fill_(0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.embedding(x)

class TextDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        texts: List[str],
        tokenizer: CustomTokenizer,
        max_length: int = 1024
    ):
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Tokenize all texts
        self.encoded_texts = [
            self.tokenizer.encode(text)[:max_length] for text in texts
        ]

    def __len__(self) -> int:
        return len(self.encoded_texts)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        tokens = self.encoded_texts[idx]

        # Create input and target sequences for language modeling
        input_ids = torch.tensor(tokens[:-1], dtype=torch.long)
        target_ids = torch.tensor(tokens[1:], dtype=torch.long)

        return input_ids, target_ids

def create_dataloader(
    texts: List[str],
    tokenizer: CustomTokenizer,
    batch_size: int = 32,
    max_length: int = 1024,
    shuffle: bool = True
) -> torch.utils.data.DataLoader:
    dataset = TextDataset(texts, tokenizer, max_length)

    # Custom collate function to handle variable length sequences
    def collate_fn(batch):
        input_ids, target_ids = zip(*batch)

        # Pad sequences
        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
        target_ids = pad_sequence(target_ids, batch_first=True, padding_value=tokenizer.pad_token_id)

        # Create attention mask
        attention_mask = (input_ids != tokenizer.pad_token_id).float()

        return {
            'input_ids': input_ids,
            'target_ids': target_ids,
            'attention_mask': attention_mask
        }

    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        collate_fn=collate_fn
    )

# Example usage
def create_custom_tokenizer_and_embedding(
    texts: List[str],
    vocab_size: int = 50000,
    embedding_dim: int = 512,
    min_freq: int = 2
):
    # Create and train tokenizer
    tokenizer = CustomTokenizer(vocab_size=vocab_size, min_freq=min_freq)
    tokenizer.train_from_texts(texts)

    # Create embedding layer
    embedding = CustomEmbedding(
        vocab_size=tokenizer.vocab_size_current,
        embedding_dim=embedding_dim,
        pad_idx=tokenizer.pad_token_id
    )

    return tokenizer, embedding

In [ ]:
class TitanModelWithCustomEmbedding(nn.Module):
    def __init__(
        self,
        embedding_layer: CustomEmbedding,
        max_seq_length: int = 1024,
        depth: int = 12,
        num_heads: int = 8,
        head_dim: int = 64,
        mlp_ratio: int = 4,
        dropout: float = 0.1
    ):
        super().__init__()
        dim = embedding_layer.embedding_dim

        self.embedding = embedding_layer
        self.pos_embedding = nn.Parameter(torch.zeros(1, max_seq_length, dim))

        self.transformer = TitanTransformer(
            dim=dim,
            depth=depth,
            num_heads=num_heads,
            head_dim=head_dim,
            mlp_ratio=mlp_ratio,
            dropout=dropout
        )

        # Add final projection for token prediction
        self.output_projection = nn.Linear(dim, embedding_layer.embedding.num_embeddings)

    def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor = None):
        # Get embeddings
        x = self.embedding(input_ids)

        # Add positional embeddings
        x = x + self.pos_embedding[:, :x.size(1), :]

        # Apply transformer
        x = self.transformer(x, attention_mask)

        # Project to vocabulary
        logits = self.output_projection(x)

        return logits

# Example usage
def create_complete_model(texts: List[str], vocab_size: int = 50000, embedding_dim: int = 512):
    # Create tokenizer and embedding
    tokenizer, embedding = create_custom_tokenizer_and_embedding(
        texts=texts,
        vocab_size=vocab_size,
        embedding_dim=embedding_dim
    )

    # Create model
    model = TitanModelWithCustomEmbedding(
        embedding_layer=embedding,
        max_seq_length=1024,
        depth=12
    )

    return model, tokenizer

In [ ]:
# Example texts for training
texts = [
    "Here is an example text.",
    "Another example for training.",
    # ... add more texts
]

# Create model and tokenizer
model, tokenizer = create_complete_model(texts)

# Create dataloader
dataloader = create_dataloader(
    texts=texts,
    tokenizer=tokenizer,
    batch_size=32
)

# Example of processing a single text
text = "Here is a test sentence."
input_ids = torch.tensor([tokenizer.encode(text)], dtype=torch.long)
attention_mask = (input_ids != tokenizer.pad_token_id).float()

# Get model output
outputs = model(input_ids, attention_mask)

In [ ]:
print(outputs)

tensor([[[ 0.1218, -1.0911, -0.2457, -0.9009, -0.7150, -0.5246],
         [ 0.1308, -1.1791,  0.0567, -1.1022, -0.6318, -0.5099],
         [ 0.0257, -0.8664,  0.0607, -1.2659, -0.5559, -0.5046],
         [ 0.0948, -1.0489,  0.0833, -1.2233, -0.3622, -0.6934],
         [ 0.3440, -1.1481, -0.0911, -1.2027, -0.4988, -0.6424],
         [ 0.1207, -1.1717, -0.1983, -1.0799, -0.4003, -0.8375],
         [ 0.1850, -0.9899, -0.0900, -1.7382, -0.5805, -0.3451],
         [ 0.3422, -1.0191, -0.2515, -1.4591, -0.9092, -0.8507]]],
       grad_fn=<ViewBackward0>)


## Creating traning pipeline

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import wandb
import os
from typing import Dict, Any
import json
from tqdm import tqdm

class TitanTrainer:
    def __init__(
        self,
        model: nn.Module,
        tokenizer: CustomTokenizer,
        train_dataloader: torch.utils.data.DataLoader,
        val_dataloader: torch.utils.data.DataLoader = None,
        learning_rate: float = 1e-4,
        warmup_steps: int = 1000,
        max_steps: int = 100000,
        gradient_clip: float = 1.0,
        device: str = 'cuda' if torch.cuda.is_available() else 'cpu',
        checkpoint_dir: str = 'checkpoints',
        use_wandb: bool = False
    ):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.train_dataloader = train_dataloader
        self.val_dataloader = val_dataloader
        self.device = device
        self.checkpoint_dir = checkpoint_dir
        self.use_wandb = use_wandb

        # Create checkpoint directory
        os.makedirs(checkpoint_dir, exist_ok=True)

        # Initialize optimizer
        self.optimizer = optim.AdamW(
            self.model.parameters(),
            lr=learning_rate,
            betas=(0.9, 0.999),
            eps=1e-8,
            weight_decay=0.01
        )

        # Initialize learning rate scheduler
        self.scheduler = CosineAnnealingLR(
            self.optimizer,
            T_max=max_steps,
            eta_min=learning_rate/100
        )

        self.warmup_steps = warmup_steps
        self.gradient_clip = gradient_clip

        # Initialize training state
        self.current_step = 0
        self.best_val_loss = float('inf')

    def _get_warmup_lr(self, step: int) -> float:
        return min(1.0, step / self.warmup_steps)

    def _compute_loss(self, logits: torch.Tensor, targets: torch.Tensor, pad_idx: int) -> torch.Tensor:
        # Reshape logits and targets for loss computation
        logits = logits.view(-1, logits.size(-1))
        targets = targets.view(-1)

        # Create mask for padding
        mask = (targets != pad_idx).float()

        # Compute cross entropy loss
        loss = F.cross_entropy(logits, targets, reduction='none')

        # Apply mask and compute mean
        loss = (loss * mask).sum() / mask.sum()

        return loss

    def train_step(self, batch: Dict[str, torch.Tensor]) -> Dict[str, float]:
        self.model.train()

        # Move batch to device
        input_ids = batch['input_ids'].to(self.device)
        target_ids = batch['target_ids'].to(self.device)
        attention_mask = batch['attention_mask'].to(self.device)

        # Forward pass
        logits = self.model(input_ids, attention_mask)

        # Compute loss
        loss = self._compute_loss(logits, target_ids, self.tokenizer.pad_token_id)

        # Backward pass
        loss.backward()

        # Clip gradients
        torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.gradient_clip)

        # Apply warmup
        if self.current_step < self.warmup_steps:
            lr_mult = self._get_warmup_lr(self.current_step)
            for param_group in self.optimizer.param_groups:
                param_group['lr'] *= lr_mult

        # Update parameters
        self.optimizer.step()
        self.scheduler.step()
        self.optimizer.zero_grad()

        return {'loss': loss.item()}

    @torch.no_grad()
    def evaluate(self) -> Dict[str, float]:
        if not self.val_dataloader:
            return {}

        self.model.eval()
        total_loss = 0
        total_tokens = 0

        for batch in self.val_dataloader:
            input_ids = batch['input_ids'].to(self.device)
            target_ids = batch['target_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)

            logits = self.model(input_ids, attention_mask)
            loss = self._compute_loss(logits, target_ids, self.tokenizer.pad_token_id)

            total_loss += loss.item() * input_ids.size(0)
            total_tokens += input_ids.size(0)

        return {'val_loss': total_loss / total_tokens}

    def save_checkpoint(self, metrics: Dict[str, float], is_best: bool = False):
        checkpoint = {
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'metrics': metrics,
            'step': self.current_step
        }

        # Save latest checkpoint
        path = os.path.join(self.checkpoint_dir, 'latest.pt')
        torch.save(checkpoint, path)

        # Save best checkpoint if needed
        if is_best:
            best_path = os.path.join(self.checkpoint_dir, 'best.pt')
            torch.save(checkpoint, best_path)

    def train(self, num_epochs: int):
        if self.use_wandb:
            wandb.init(project="titan-transformer")

        for epoch in range(num_epochs):
            epoch_metrics = {'epoch': epoch}

            # Training loop
            pbar = tqdm(self.train_dataloader, desc=f'Epoch {epoch}')
            for batch in pbar:
                metrics = self.train_step(batch)
                epoch_metrics.update(metrics)

                # Update progress bar
                pbar.set_postfix(loss=f"{metrics['loss']:.4f}")

                self.current_step += 1

                # Log metrics
                if self.use_wandb:
                    wandb.log(metrics, step=self.current_step)

            # Validation
            val_metrics = self.evaluate()
            epoch_metrics.update(val_metrics)

            # Check for best model
            if val_metrics and val_metrics['val_loss'] < self.best_val_loss:
                self.best_val_loss = val_metrics['val_loss']
                self.save_checkpoint(epoch_metrics, is_best=True)

            # Save regular checkpoint
            self.save_checkpoint(epoch_metrics)

            # Log epoch metrics
            if self.use_wandb:
                wandb.log(epoch_metrics, step=self.current_step)

            print(f"Epoch {epoch} metrics:", epoch_metrics)


In [ ]:
train_texts = [
    "The quick brown fox jumps over the lazy dog.",
    "Artificial Intelligence is transforming the world.",
    "Python is a popular programming language.",
    "Machine learning requires large datasets for training.",
    "The sky is clear and blue today.",
    "Natural language processing enables machines to understand text.",
    "Self-driving cars use advanced sensors and algorithms.",
    "Data science combines statistics and computer science.",
    "Reinforcement learning involves agents taking actions in environments.",
    "Deep learning utilizes neural networks for predictions.",
    "Quantum computing is a rapidly emerging field.",
    "Cloud computing offers scalable storage solutions.",
    "IoT devices are interconnected over networks.",
    "Cybersecurity is crucial in today's digital era.",
    "The sun rises in the east and sets in the west.",
    "Renewable energy sources are vital for sustainability.",
    "Blockchain technology ensures secure transactions.",
    "Mathematics is the foundation of many sciences.",
    "Artificial neural networks mimic the human brain.",
    "The universe is vast and mysterious."
]
val_texts = [
    "Graphs are used to represent relationships between entities.",
    "Programming requires logical thinking and problem-solving skills.",
    "AI ethics is an important area of research.",
    "The weather today is perfect for a picnic.",
    "Virtual reality creates immersive digital experiences.",
    "Big data analytics helps businesses make better decisions.",
    "Space exploration expands our understanding of the cosmos.",
    "Autonomous drones are used in agriculture and delivery.",
    "Data visualization simplifies complex data insights.",
    "The human genome holds the secrets of biology."
]
test_texts = [
    "Robotics integrates mechanics and AI for automation.",
    "Digital transformation accelerates technological adoption.",
    "Text-to-speech models improve accessibility.",
    "The internet connects people across the globe.",
    "Energy-efficient algorithms are vital for green computing."
]


In [ ]:
# Create dataloaders
train_dataloader = create_dataloader(train_texts, tokenizer, batch_size=32)
val_dataloader = create_dataloader(val_texts, tokenizer, batch_size=32)

# Initialize trainer
trainer = TitanTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    learning_rate=1e-4,
    use_wandb=True  # Set to True if using Weights & Biases
)

# Train model
trainer.train(num_epochs=10)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 0: 100%|██████████| 1/1 [00:09<00:00,  9.95s/it, loss=2.3673]


...

Epoch 7 metrics: {'epoch': 7, 'loss': 2.3485639095306396, 'val_loss': 2.359609603881836}


Epoch 8: 100%|██████████| 1/1 [00:05<00:00,  5.68s/it, loss=2.3788]


Epoch 8 metrics: {'epoch': 8, 'loss': 2.3788001537323, 'val_loss': 2.359609365463257}


Epoch 9: 100%|██████████| 1/1 [00:06<00:00,  6.99s/it, loss=2.3707]


Epoch 9 metrics: {'epoch': 9, 'loss': 2.3706822395324707, 'val_loss': 2.359609365463257}


## torch server handler class

In [ ]:
# TorchServe handler class
class TitanHandler:
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def initialize(self, context):
        """Initialize model and tokenizer."""
        properties = context.system_properties
        model_dir = properties.get('model_dir')

        # Load tokenizer
        with open(os.path.join(model_dir, 'tokenizer_config.json'), 'r') as f:
            tokenizer_config = json.load(f)
        self.tokenizer = CustomTokenizer(**tokenizer_config)
        self.tokenizer.load_vocab(os.path.join(model_dir, 'vocab.txt'))

        # Load model
        checkpoint = torch.load(
            os.path.join(model_dir, 'model.pt'),
            map_location=self.device
        )

        # Initialize model (you'll need to match the configuration used during training)
        self.model = TitanModelWithCustomEmbedding(
            embedding_layer=CustomEmbedding(
                vocab_size=self.tokenizer.vocab_size_current,
                embedding_dim=512  # Make sure this matches your training config
            )
        )
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.to(self.device)
        self.model.eval()

    def preprocess(self, data):
        """Preprocess input text."""
        text = data[0].get('body').decode('utf-8')

        # Tokenize input
        input_ids = torch.tensor(
            [self.tokenizer.encode(text)],
            dtype=torch.long
        ).to(self.device)

        attention_mask = (input_ids != self.tokenizer.pad_token_id).float()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }

    def inference(self, data):
        """Run inference on processed input."""
        with torch.no_grad():
            outputs = self.model(
                data['input_ids'],
                data['attention_mask']
            )
        return outputs

    def postprocess(self, inference_output):
        """Process model output to response format."""
        # Get predicted tokens
        predictions = inference_output.argmax(dim=-1)

        # Decode tokens to text
        response = self.tokenizer.decode(predictions[0].tolist())

        return [{'generated_text': response}]



TypeError: prepare_model_for_serving() takes 2 positional arguments but 3 were given

In [ ]:
import os
import json
import torch


def prepare_model_for_serving(model_path: str, save_dir: str, tokenizer: CustomTokenizer):
    """
    Prepare model artifacts for TorchServe.

    Args:
        model_path (str): Path to the saved model checkpoint.
        save_dir (str): Directory to save the prepared model artifacts.
        tokenizer (CustomTokenizer): An instance of the tokenizer to save configuration and vocabulary.
    """
    os.makedirs(save_dir, exist_ok=True)

    # Load the model from the checkpoint
    model = torch.load(model_path)

    # Save the model in the specified directory
    torch.save(model, os.path.join(save_dir, 'model.pt'))

    # Prepare tokenizer configuration
    tokenizer_config = {
        'vocab_size': tokenizer.vocab_size,
        'min_freq': tokenizer.min_freq,
        'special_tokens': tokenizer.special_tokens,
    }

    # Save tokenizer configuration as JSON
    with open(os.path.join(save_dir, 'tokenizer_config.json'), 'w') as f:
        json.dump(tokenizer_config, f)

    # Save tokenizer vocabulary
    tokenizer.save_vocab(os.path.join(save_dir, 'vocab.txt'))


# Example usage
# Create an instance of CustomTokenizer
tokenizer = CustomTokenizer(
    vocab_size=50000,
    min_freq=2,
    special_tokens=['<pad>', '<eos>', '<bos>', '<unk>']
)

# Call the function with the correct parameters
prepare_model_for_serving('checkpoints/best.pt', 'model_store', tokenizer)


<ipython-input-43-06d5ff2de07d>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


In [ ]:
!pip install torchserve torch-model-archiver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 8.0 MB/s eta 0:00:00


In [ ]:
handler_code = """
# TorchServe handler class
class TitanHandler:
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def initialize(self, context):
        properties = context.system_properties
        model_dir = properties.get('model_dir')

        # Load tokenizer
        with open(os.path.join(model_dir, 'tokenizer_config.json'), 'r') as f:
            tokenizer_config = json.load(f)
        self.tokenizer = CustomTokenizer(**tokenizer_config)
        self.tokenizer.load_vocab(os.path.join(model_dir, 'vocab.txt'))

        # Load model
        checkpoint = torch.load(
            os.path.join(model_dir, 'model.pt'),
            map_location=self.device
        )

        # Initialize model (you'll need to match the configuration used during training)
        self.model = TitanModelWithCustomEmbedding(
            embedding_layer=CustomEmbedding(
                vocab_size=self.tokenizer.vocab_size_current,
                embedding_dim=512  # Make sure this matches your training config
            )
        )
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.to(self.device)
        self.model.eval()

    def preprocess(self, data):
        text = data[0].get('body').decode('utf-8')

        # Tokenize input
        input_ids = torch.tensor(
            [self.tokenizer.encode(text)],
            dtype=torch.long
        ).to(self.device)

        attention_mask = (input_ids != self.tokenizer.pad_token_id).float()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }

    def inference(self, data):
        with torch.no_grad():
            outputs = self.model(
                data['input_ids'],
                data['attention_mask']
            )
        return outputs

    def postprocess(self, inference_output):
        # Get predicted tokens
        predictions = inference_output.argmax(dim=-1)

        # Decode tokens to text
        response = self.tokenizer.decode(predictions[0].tolist())

        return [{'generated_text': response}]


"""

# Save it to a file
with open("titan_handler.py", "w") as f:
    f.write(handler_code)

In [ ]:
!torch-model-archiver --model-name titan \
                      --version 1.0 \
                      --model-file model_store/model.pt \
                      --handler titan_handler.py \
                      --extra-files "model_store/tokenizer_config.json,model_store/vocab.txt" \
                      --export-path model_store \
                      --force


In [ ]:
!lsof -i :8080


COMMAND PID USER   FD   TYPE  DEVICE SIZE/OFF NODE NAME
node      6 root   21u  IPv6   19939      0t0  TCP *:8080 (LISTEN)
node      6 root   26u  IPv6 1016000      0t0  TCP 17c4989733c5:8080->172.28.0.1:39786 (ESTABLISHED)
node      6 root   28u  IPv6 1017108      0t0  TCP 17c4989733c5:8080->172.28.0.1:45550 (ESTABLISHED)
node      6 root   29u  IPv6   25417      0t0  TCP 17c4989733c5:8080->172.28.0.1:55124 (ESTABLISHED)
node      6 root   31u  IPv6  867221      0t0  TCP 17c4989733c5:8080->172.28.0.1:50646 (ESTABLISHED)


In [ ]:
!torchserve --start --model-store model_store --models titan=titan.mar --inference-address http://127.0.0.1:8083


usage: torchserve [-h] [-v | --start | --stop] [--ts-config TS_CONFIG] [--model-store MODEL_STORE]
                  [--workflow-store WORKFLOW_STORE]
                  [--models MODEL_PATH1 MODEL_NAME=MODEL_PATH2... [MODEL_PATH1 MODEL_NAME=MODEL_PATH2... ...]]
                  [--log-config LOG_CONFIG] [--cpp-log-config CPP_LOG_CONFIG] [--foreground]
                  [--no-config-snapshots] [--plugins-path PLUGINS_PATH] [--disable-token-auth]
                  [--enable-model-api]
torchserve: error: unrecognized arguments: --inference-address http://127.0.0.1:8083


In [ ]:
!mkdir -p ~/.torchserve
!nano ~/.torchserve/config.properties


/bin/bash: line 1: nano: command not found


## push model to huggingface

In [ ]:
from huggingface_hub import login

login()


In [ ]:
from huggingface_hub import create_repo

create_repo("titan-transformer", private=False)  # Set `private=True` if you want the model to be private


RepoUrl('https://huggingface.co/rajveer43/titan-transformer', endpoint='https://huggingface.co', repo_type='model', repo_id='rajveer43/titan-transformer')

In [ ]:
from huggingface_hub import upload_file

# Upload model.pt
upload_file(
    path_or_fileobj="model_store/model.pt",
    path_in_repo="pytorch_model.bin",  # This is the standard filename for the model weights
    repo_id="rajveer43/titan-transformer"  # Replace with your Hugging Face username and repo name
)

# Upload tokenizer_config.json
upload_file(
    path_or_fileobj="model_store/tokenizer_config.json",
    path_in_repo="tokenizer_config.json",
    repo_id="rajveer43/titan-transformer"
)

# Upload vocab.txt
upload_file(
    path_or_fileobj="model_store/vocab.txt",
    path_in_repo="vocab.txt",
    repo_id="rajveer43/titan-transformer"
)


model.pt:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rajveer43/titan-transformer/commit/aaebf58cbe297a34eb4d9d0779cf4c9e4f4a2d16', commit_message='Upload vocab.txt with huggingface_hub', commit_description='', oid='aaebf58cbe297a34eb4d9d0779cf4c9e4f4a2d16', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rajveer43/titan-transformer', endpoint='https://huggingface.co', repo_type='model', repo_id='rajveer43/titan-transformer'), pr_revision=None, pr_num=None)

In [ ]:
from huggingface_hub import upload_file

# Upload model checkpoint
upload_file(
    path_or_fileobj="/content/checkpoints/best.pt",
    path_in_repo="model/checkpoints/best.pt",
    repo_id="rajveer43/titan-transformer",
    repo_type="model"
)
upload_file(
    path_or_fileobj="/content/checkpoints/latest.pt",
    path_in_repo="model/checkpoints/latest.pt",
    repo_id="rajveer43/titan-transformer",
    repo_type="model"
)


best.pt:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

latest.pt:   0%|          | 0.00/1.78G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rajveer43/titan-transformer/commit/4a7583a6d08169c804d0a4b3bf2b96ba990d798c', commit_message='Upload model/checkpoints/latest.pt with huggingface_hub', commit_description='', oid='4a7583a6d08169c804d0a4b3bf2b96ba990d798c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rajveer43/titan-transformer', endpoint='https://huggingface.co', repo_type='model', repo_id='rajveer43/titan-transformer'), pr_revision=None, pr_num=None)

In [ ]:
import os
from huggingface_hub import upload_file

logs_folder = '/content/logs'  # Path to your logs folder
repo_id = "rajveer43/titan-transformer"  # Your repo name on Hugging Face

# Loop through all files in the logs folder and upload them
for root, dirs, files in os.walk(logs_folder):
    for file in files:
        file_path = os.path.join(root, file)
        # Upload each file to the Hugging Face repo
        upload_file(
            path_or_fileobj=file_path,
            path_in_repo=f"logs/{file}",  # Keep the folder structure in the repo
            repo_id=repo_id,
            repo_type="model"
        )


In [59]:
import os
from huggingface_hub import upload_file

logs_folder = '/content/wandb'  # Path to your logs folder
repo_id = "rajveer43/titan-transformer"  # Your repo name on Hugging Face

# Loop through all files in the logs folder and upload them
for root, dirs, files in os.walk(logs_folder):
    for file in files:
        file_path = os.path.join(root, file)
        # Upload each file to the Hugging Face repo
        upload_file(
            path_or_fileobj=file_path,
            path_in_repo=f"wandb/{file}",  # Keep the folder structure in the repo
            repo_id=repo_id,
            repo_type="model"
        )


No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.
